In [ ]:
import pandas as pd

In [ ]:
import sys

In [ ]:
sys.path.append("/fs/nas14/other0/cbenitez/Pkgs/")

In [ ]:
import MT as mt

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
import emcee3 as mc

In [ ]:
import corner

__________________

In [ ]:
def vb_model(c,theta):
    A,B,C,K = theta
    #return -c[0]*np.sin(c[2])*(K+C*np.cos(2*c[1])+A*np.sin(2*c[1]))+v_LSR[0]*np.sin(c[2])*np.cos(c[1])+v_LSR[1]*np.sin(c[2])*np.sin(c[1])
    return -c[0]*np.sin(c[2])*np.cos(c[2])*(K+C*np.cos(2*c[1])+A*np.sin(2*c[1]))
def vell_model(c,theta):
    A,B,C,K = theta
    #return c[0]*np.cos(c[2])*(B+A*np.cos(2*c[1])-C*np.sin(2*c[1]))-v_LSR[0]*np.sin(c[1])-v_LSR[1]*np.cos(c[1])
    return c[0]*np.cos(c[2])*(B+A*np.cos(2*c[1])-C*np.sin(2*c[1]))

In [ ]:
def cart2galactic(coord,coord_sun):
    zh = np.array([0.,0.,1.])
    Xh_new = -coord_sun/np.linalg.norm(coord_sun)
    Yh_new = np.cross(zh,Xh_new)
    dr = coord-coord_sun
    drn = np.linalg.norm(dr)
    ell = np.arctan2(Yh_new.dot(dr),Xh_new.dot(dr))
    b = np.arcsin((coord[2]-coord_sun[2])/drn)
    return np.array([drn,ell,b])

In [ ]:
def vcart2vgalactic(coord,coord_sun,vel,vel_sun):
    dv = vel-vel_sun
    #dv = vel
    dr = coord-coord_sun
    v_r = dv.dot(dr)/np.linalg.norm(dr)
    v_ell = np.cross(dv,dr)[2]/np.linalg.norm(dr[:2:])
    v_b = -(dr[2]*(dr[0]*dv[0]+dr[1]*dv[1])-(dr[0]**2+dr[1]**2)*dv[2])/(np.linalg.norm(dr)**2*np.sqrt(dr[0]**2+dr[1]**2))
    return np.array([v_r, v_ell, v_b])

In [ ]:
plt.plot([.1/(np.linalg.norm(c-r_LSR)+.1) for c in np.array(can211[['x','y','z']])])

In [ ]:
def vcart2vgalactic_exp(coord,coord_sun,vel,vel_sun,mod):
    dv = vel-vel_sun
    #dv = vel
    dr = coord-coord_sun
    v_r = dv.dot(dr)/np.linalg.norm(dr)+mod/(np.linalg.norm(dr)+.1)
    v_ell = np.cross(dv,dr)[2]/np.linalg.norm(dr[:2:])
    v_b = -(dr[2]*(dr[0]*dv[0]+dr[1]*dv[1])-(dr[0]**2+dr[1]**2)*dv[2])/(np.linalg.norm(dr)**2*np.sqrt(dr[0]**2+dr[1]**2))
    return np.array([v_r, v_ell, v_b])

_______

In [ ]:
can,lsr = mt.read_dat(mt.tardir+'canicas_geoamr/','canicas_geoamr','canicas_geoamrLSR')

In [ ]:
ec = 2
tc = 11

fig = plt.figure(figsize=(9,9))

ax = fig.add_subplot(2,2,1)
ax.scatter(can.x[(can.edad==ec) & (can.tiempo==tc)],can.y[(can.edad==ec) & (can.tiempo==tc)],s=4)
ax.scatter([7.4],[-.2],c='r')
ax.set_xlabel(r'$x \mathrm{[kpc]}$',fontsize=14)
ax.set_ylabel(r'$y \mathrm{[kpc]}$',fontsize=14)
ax.set_xlim(3,8)
ax.set_ylim(-2.5,2.5)

ax = fig.add_subplot(2,2,2)
ax.scatter(can.x[(can.edad==ec) & (can.tiempo==tc)],can.z[(can.edad==ec) & (can.tiempo==tc)],s=4)
ax.set_xlabel(r'$x \mathrm{[kpc]}$',fontsize=14)
ax.set_ylabel(r'$z \mathrm{[kpc]}$',fontsize=14)
ax.set_xlim(3,8)
ax.set_ylim(-2.5,2.5)

ax = fig.add_subplot(2,2,3)
ax.scatter(can.y[(can.edad==ec) & (can.tiempo==tc)],can.z[(can.edad==ec) & (can.tiempo==tc)],s=4)
ax.set_xlabel(r'$y \mathrm{[kpc]}$',fontsize=14)
ax.set_ylabel(r'$z \mathrm{[kpc]}$',fontsize=14)
ax.set_xlim(-2.5,2.5)
ax.set_ylim(-2.5,2.5)

ax = fig.add_subplot(2, 2, 4, projection='3d')
ax.scatter(np.array(can.x[(can.edad==ec) & (can.tiempo==tc)]),\
           np.array(can.y[(can.edad==ec) & (can.tiempo==tc)]),\
           np.array(can.z[(can.edad==ec) & (can.tiempo==tc)]),\
           c = np.array(can.dens[(can.edad==ec) & (can.tiempo==tc)]), alpha=.3, marker='o')
ax.set_xlabel(r'$x \mathrm{[kpc]}$',fontsize=10)
ax.set_ylabel(r'$y \mathrm{[kpc]}$',fontsize=10)
ax.set_zlabel(r'$z \mathrm{[kpc]}$',fontsize=10)
ax.set_xlim(3,8)
ax.set_ylim(-2.5,2.5)
ax.set_zlim(-2.5,2.5)

plt.show()


In [ ]:
can211 = can[(can.edad==2) & (can.tiempo==11)].copy()

In [ ]:
can211[['vx','vy','vz']] = can211[['vx','vy','vz']]*30856802500000000.00

In [ ]:
#r_LSR = np.array([6.8,-1.3,0]) #3,17
r_LSR = np.array([7.4,-.2,0.]) #2,11
v_LSR = mt.vel_circ(r_LSR,mt.force_T,)*30856802500000000.00 
#v_LSR = np.array(np.sqrt(abs(np.dot(mt.force_T(r_LSR[0],r_LSR[1],r_LSR[2]),[r_LSR[0],r_LSR[1],r_LSR[2]])))*np.array([r_LSR[1],-r_LSR[0],0.0])/np.linalg.norm(r_LSR))*30856802500000000.00 

In [ ]:
#vv_galac = [vcart2vgalactic(np.array(can211[['x','y','z']])[i],r_LSR,np.array(can211[['vx','vy','vz']])[i],v_LSR) for i in range(len(can211))]  
vv_circ_cart = np.array([mt.vel_circ(c,mt.force_T,)*30856802500000000.00 for c in np.array(can211[['x','y','z']])])
cc_cart = np.array(can211[['x','y','z']])
vv_galac = np.array([vcart2vgalactic(cc_cart[i],r_LSR,vv_circ_cart[i],v_LSR) for i in range(len(vv_circ_cart))])
cc_galac = np.array([cart2galactic(c,r_LSR) for c in cc_cart])

In [ ]:
ll = np.linspace(-np.pi,np.pi,500)

fig, axs = plt.subplots(3, 1, sharex=True,figsize=(9,9))
fig.subplots_adjust(hspace=0)

axs[0].scatter([c[1] for c in cc_galac],[v[0] for v in vv_galac],s=4)
axs[1].scatter([c[1] for c in cc_galac],[v[1] for v in vv_galac],s=4)
axs[2].scatter([c[1] for c in cc_galac],[v[2] for v in vv_galac],s=4)

#axs[0].set_yticks(np.arange(-0.9, 1.0, 0.4))
#axs[0].set_ylim(-1, 1)

plt.xlabel(r'$\ell$',fontsize=14)
axs[0].set_ylabel(r'$v_{r}$',fontsize=14)
axs[1].set_ylabel(r'$v_{\ell}$',fontsize=14)
axs[2].set_ylabel(r'$v_{b}$',fontsize=14)

plt.show()

In [ ]:
ll = np.linspace(-np.pi,np.pi,500)

fig, axs = plt.subplots(3, 1, sharex=True,figsize=(9,9))
fig.subplots_adjust(hspace=0)

axs[0].scatter([c[1] for c in cc_galac],[vv_galac[i][0]/cc_galac[i][0]/np.cos(cc_galac[i][2]) for i in range(len(vv_galac))],s=4)
axs[1].scatter([c[1] for c in cc_galac],[vv_galac[i][1]/cc_galac[i][0]/np.cos(cc_galac[i][2]) for i in range(len(vv_galac))],s=4)
axs[2].scatter([c[1] for c in cc_galac],[vv_galac[i][2]/cc_galac[i][0]/np.sin(cc_galac[i][2])/np.cos(cc_galac[i][2]) for i in range(len(vv_galac))],s=4)

#axs[0].set_yticks(np.arange(-0.9, 1.0, 0.4))
#axs[0].set_ylim(-1, 1)

plt.xlabel(r'$\ell$',fontsize=14)
axs[0].set_ylabel(r'$\frac{v_{r}}{r\cos{b}}$',fontsize=14)
axs[1].set_ylabel(r'$\Delta\mu_{\ell}\,\,\, \left(\frac{v_{\ell}}{r\cos{b}} \right)$',fontsize=14)
axs[2].set_ylabel(r'$\Delta\mu_b \,\,\, \left(\frac{v_{b}}{r\sin{b}\cos{b}}\right)$',fontsize=14)
axs[2].set_ylim(-150,150)
plt.show()

________

$$\ln \mathcal{L}=-\frac{1}{2} \sum_{i}\left(\frac{\left[v_{\ell}^{i}-v_{\ell}\left(r_i,\ell_{i}, b_{i}\right)\right]^{2}}{\sigma_{v\ell}^{2}}+\ln \sigma_{v \ell}^{2}+\frac{\left[v_{b}^{i}-v_{b}\left(r_i,\ell_{i}, b_{i}\right)\right]^{2}}{\sigma_{v b}^{2}}+\ln \sigma_{v_{b}}^{2}\right)$$

In [ ]:
def log_likelihood(theta, coords, vels):
    #A,B,C,K = theta
    velleq = [vell_model(c,theta) for c in coords]
    vbeq = [vb_model(c,theta) for c in coords]
    vvell = np.array([v[1] for v in vels])
    vvb = np.array([v[2] for v in vels])
    #sigma2 = dveq ** 2 * np.array([np.exp(2*logf1),np.exp(2*logf2)])
    return -0.5 * np.sum(np.sqrt((vvell - velleq)** 2) +np.sqrt((vvb - vbeq)** 2) )

In [ ]:
def log_prior(theta):
    A,B,C,K = theta
    if -20.0 < A < 20 and -20 < B < 20.0 and -20.0 < C < 20.0 and -20.<K<20.:
        return 0.0
    return -np.inf
    #return 0.0

In [ ]:
def log_probability(theta, coords, vels):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, coords, vels)

In [ ]:
from scipy.optimize import minimize

In [ ]:
np.random.seed(42)
nll = lambda *args: -log_likelihood(*args)
initial = np.array([0.,0.,0.,0.])
soln = minimize(nll, initial,method = 'Nelder-Mead', args=(cc_galac, vv_galac))

In [ ]:
soln

In [ ]:
pos = soln.x + 1e-4 * np.random.randn(50, 4)
nwalkers, ndim = pos.shape

sampler = mc.EnsembleSampler(nwalkers, ndim, log_probability, args=(cc_galac, vv_galac))
sampler.run_mcmc(pos, 1000,progress=True);

In [ ]:
fig, axes = plt.subplots(4, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["A", "B", "C",'K']
for i in range(4):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
flat_samples = sampler.get_chain(discard=50, thin=15, flat=True)

fig = corner.corner(
    flat_samples,  labels=labels
);

In [ ]:
mcmc = [np.percentile(flat_samples[:, i], [16, 50, 84]) for i in range(ndim)]
qq = [np.diff(mcmc[i]) for i in range(ndim)]
theta_f = [par[1] for par in mcmc]

In [ ]:
ll = np.linspace(-np.pi,np.pi,500)

fig, axs = plt.subplots(2, 1, sharex=True,figsize=(9,9))
fig.subplots_adjust(hspace=0)

axs[0].plot(ll,[vell_model(c,theta_f)/.4/np.cos(.35) for c in [np.array([.4,l,.35]) for l in ll]])
axs[0].scatter([c[1] for c in cc_galac],[vv_galac[i][1]/cc_galac[i][0]/np.cos(cc_galac[i][2]) for i in range(len(vv_galac))],s=4)
axs[1].plot(ll,[-vb_model(c,theta_f)/.4/np.cos(.35)/np.sin(.35) for c in [np.array([.4,l,.35]) for l in ll]])
axs[1].scatter([c[1] for c in cc_galac],[vv_galac[i][2]/cc_galac[i][0]/np.sin(cc_galac[i][2])/np.cos(cc_galac[i][2]) for i in range(len(vv_galac))],s=4)

#axs[0].set_yticks(np.arange(-0.9, 1.0, 0.4))
#axs[0].set_ylim(-1, 1)

plt.xlabel(r'$\ell$',fontsize=14)
axs[0].set_ylabel(r'$\Delta\mu_{\ell}\,\,\, \left(\frac{v_{\ell}}{r\cos{b}} \right)$',fontsize=14)
axs[1].set_ylabel(r'$\Delta\mu_b \,\,\, \left(\frac{v_{b}}{r\sin{b}}\right)$',fontsize=14)
axs[1].set_ylim(-150,150)
plt.show()

In [ ]:
theta_f

____________

## Con ruido

In [ ]:
vv_cruido = vv_galac * (1. + .2*np.random.rand(len(vv_galac),3) -.1)

In [ ]:
ll = np.linspace(-np.pi,np.pi,500)
vv = vv_cruido
fig, axs = plt.subplots(3, 1, sharex=True,figsize=(9,9))
fig.subplots_adjust(hspace=0)

axs[0].scatter([c[1] for c in cc_galac],[v[0] for v in vv],s=4,label='con ruido')
axs[1].scatter([c[1] for c in cc_galac],[v[1] for v in vv],s=4)
axs[2].scatter([c[1] for c in cc_galac],[v[2] for v in vv],s=4)
axs[0].scatter([c[1] for c in cc_galac],[v[0] for v in vv_galac],s=4,c='orange',label='sin ruido')
axs[1].scatter([c[1] for c in cc_galac],[v[1] for v in vv_galac],s=4,c='orange')
axs[2].scatter([c[1] for c in cc_galac],[v[2] for v in vv_galac],s=4,c='orange')

#axs[0].set_yticks(np.arange(-0.9, 1.0, 0.4))
#axs[0].set_ylim(-1, 1)

plt.xlabel(r'$\ell$',fontsize=14)
axs[0].set_ylabel(r'$v_{r}$',fontsize=14)
axs[1].set_ylabel(r'$v_{\ell}$',fontsize=14)
axs[2].set_ylabel(r'$v_{b}$',fontsize=14)

axs[0].legend()

plt.show()

In [ ]:
ll = np.linspace(-np.pi,np.pi,500)

fig, axs = plt.subplots(3, 1, sharex=True,figsize=(9,9))
fig.subplots_adjust(hspace=0)

axs[0].scatter([c[1] for c in cc_galac],[vv[i][0]/cc_galac[i][0]/np.cos(cc_galac[i][2]) for i in range(len(vv))],s=4)
axs[1].scatter([c[1] for c in cc_galac],[vv[i][1]/cc_galac[i][0]/np.cos(cc_galac[i][2]) for i in range(len(vv))],s=4)
axs[2].scatter([c[1] for c in cc_galac],[vv[i][2]/cc_galac[i][0]/np.sin(cc_galac[i][2])/np.cos(cc_galac[i][2]) for i in range(len(vv_galac))],s=4)

#axs[0].set_yticks(np.arange(-0.9, 1.0, 0.4))
#axs[0].set_ylim(-1, 1)

plt.xlabel(r'$\ell$',fontsize=14)
axs[0].set_ylabel(r'$\frac{v_{r}}{r\cos{b}}$',fontsize=14)
axs[1].set_ylabel(r'$\Delta\mu_{\ell}\,\,\, \left(\frac{v_{\ell}}{r\cos{b}} \right)$',fontsize=14)
axs[2].set_ylabel(r'$\Delta\mu_b \,\,\, \left(\frac{v_{b}}{r\sin{b}\cos{b}}\right)$',fontsize=14)
axs[2].set_ylim(-150,150)
plt.show()

In [ ]:
def walk(vv):
    np.random.seed(42)
    nll = lambda *args: -log_likelihood(*args)
    initial = np.array([0.,0.,0.,0.])
    soln = minimize(nll, initial,method = 'Nelder-Mead', args=(cc_galac, vv))
    pos = soln.x + 1e-4 * np.random.randn(50, 4)
    nwalkers, ndim = pos.shape

    sampler = mc.EnsembleSampler(nwalkers, ndim, log_probability, args=(cc_galac, vv_galac))
    sampler.run_mcmc(pos, 1000,progress=True);
    
    mcmc = [np.percentile(flat_samples[:, i], [16, 50, 84]) for i in range(ndim)]
    qq = [np.diff(mcmc[i]) for i in range(ndim)]
    return [par[1] for par in mcmc], mcmc, qq, sampler

In [ ]:
cam = walk(vv_cruido)

In [ ]:
cam = cam_ruido

In [ ]:
fig, axes = plt.subplots(4, figsize=(10, 7), sharex=True)
samples = cam[3].get_chain()
labels = ["A", "B", "C",'K']
for i in range(4):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
flat_samples = cam[3].get_chain(discard=50, thin=15, flat=True)
fig = corner.corner(
    flat_samples,  labels=labels
);

In [ ]:
fig, axs = plt.subplots(2, 1, sharex=True,figsize=(9,9))
fig.subplots_adjust(hspace=0)

axs[0].plot(ll,[vell_model(c,cam[0])/.4/np.cos(.35) for c in [np.array([.4,l,.35]) for l in ll]])
axs[0].scatter([c[1] for c in cc_galac],[vv[i][1]/cc_galac[i][0]/np.cos(cc_galac[i][2]) for i in range(len(vv_galac))],s=4)
axs[1].plot(ll,[-vb_model(c,cam[0])/.4/np.cos(.35)/np.sin(.35) for c in [np.array([.4,l,.35]) for l in ll]])
axs[1].scatter([c[1] for c in cc_galac],[vv[i][2]/cc_galac[i][0]/np.sin(cc_galac[i][2])/np.cos(cc_galac[i][2]) for i in range(len(vv_galac))],s=4)

#axs[0].set_yticks(np.arange(-0.9, 1.0, 0.4))
#axs[0].set_ylim(-1, 1)

plt.xlabel(r'$\ell$',fontsize=14)
axs[0].set_ylabel(r'$\Delta\mu_{\ell}\,\,\, \left(\frac{v_{\ell}}{r\cos{b}} \right)$',fontsize=14)
axs[1].set_ylabel(r'$\Delta\mu_b \,\,\, \left(\frac{v_{b}}{r\sin{b}}\right)$',fontsize=14)
axs[1].set_ylim(-150,150)
plt.show()

In [ ]:
cam[0]

__________

## Con expansión

In [ ]:
vv_cexp = np.array([vcart2vgalactic_exp(cc_cart[i],r_LSR,vv_circ_cart[i],v_LSR,.1) for i in range(len(vv_circ_cart))])

In [ ]:
ll = np.linspace(-np.pi,np.pi,500)
vv = vv_cexp
fig, axs = plt.subplots(3, 1, sharex=True,figsize=(9,9))
fig.subplots_adjust(hspace=0)

axs[0].scatter([c[1] for c in cc_galac],[v[0] for v in vv],s=4,label='con ruido')
axs[1].scatter([c[1] for c in cc_galac],[v[1] for v in vv],s=4)
axs[2].scatter([c[1] for c in cc_galac],[v[2] for v in vv],s=4)
axs[0].scatter([c[1] for c in cc_galac],[v[0] for v in vv_galac],s=4,c='orange',label='sin ruido')
axs[1].scatter([c[1] for c in cc_galac],[v[1] for v in vv_galac],s=4,c='orange')
axs[2].scatter([c[1] for c in cc_galac],[v[2] for v in vv_galac],s=4,c='orange')

#axs[0].set_yticks(np.arange(-0.9, 1.0, 0.4))
#axs[0].set_ylim(-1, 1)

plt.xlabel(r'$\ell$',fontsize=14)
axs[0].set_ylabel(r'$v_{r}$',fontsize=14)
axs[1].set_ylabel(r'$v_{\ell}$',fontsize=14)
axs[2].set_ylabel(r'$v_{b}$',fontsize=14)

axs[0].legend()

plt.show()

In [ ]:
ll = np.linspace(-np.pi,np.pi,500)

fig, axs = plt.subplots(3, 1, sharex=True,figsize=(9,9))
fig.subplots_adjust(hspace=0)

axs[0].scatter([c[1] for c in cc_galac],[vv[i][0]/cc_galac[i][0]/np.cos(cc_galac[i][2]) for i in range(len(vv))],s=4)
axs[1].scatter([c[1] for c in cc_galac],[vv[i][1]/cc_galac[i][0]/np.cos(cc_galac[i][2]) for i in range(len(vv))],s=4)
axs[2].scatter([c[1] for c in cc_galac],[vv[i][2]/cc_galac[i][0]/np.sin(cc_galac[i][2])/np.cos(cc_galac[i][2]) for i in range(len(vv_galac))],s=4)

#axs[0].set_yticks(np.arange(-0.9, 1.0, 0.4))
#axs[0].set_ylim(-1, 1)

plt.xlabel(r'$\ell$',fontsize=14)
axs[0].set_ylabel(r'$\frac{v_{r}}{r\cos{b}}$',fontsize=14)
axs[1].set_ylabel(r'$\Delta\mu_{\ell}\,\,\, \left(\frac{v_{\ell}}{r\cos{b}} \right)$',fontsize=14)
axs[2].set_ylabel(r'$\Delta\mu_b \,\,\, \left(\frac{v_{b}}{r\sin{b}\cos{b}}\right)$',fontsize=14)
axs[2].set_ylim(-150,150)
plt.show()

In [ ]:
cam2 = walk(vv_cruido)

In [ ]:
fig, axes = plt.subplots(4, figsize=(10, 7), sharex=True)
samples = cam2[3].get_chain()
labels = ["A", "B", "C",'K']
for i in range(4):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
flat_samples = cam2[3].get_chain(discard=50, thin=15, flat=True)
fig = corner.corner(
    flat_samples,  labels=labels
);

In [ ]:
fig, axs = plt.subplots(2, 1, sharex=True,figsize=(9,9))
fig.subplots_adjust(hspace=0)

axs[0].plot(ll,[vell_model(c,cam2[0])/.4/np.cos(.35) for c in [np.array([.4,l,.35]) for l in ll]])
axs[0].scatter([c[1] for c in cc_galac],[vv[i][1]/cc_galac[i][0]/np.cos(cc_galac[i][2]) for i in range(len(vv_galac))],s=4)
axs[1].plot(ll,[-vb_model(c,cam2[0])/.4/np.cos(.35)/np.sin(.35) for c in [np.array([.4,l,.35]) for l in ll]])
axs[1].scatter([c[1] for c in cc_galac],[vv[i][2]/cc_galac[i][0]/np.sin(cc_galac[i][2])/np.cos(cc_galac[i][2]) for i in range(len(vv_galac))],s=4)

#axs[0].set_yticks(np.arange(-0.9, 1.0, 0.4))
#axs[0].set_ylim(-1, 1)

plt.xlabel(r'$\ell$',fontsize=14)
axs[0].set_ylabel(r'$\Delta\mu_{\ell}\,\,\, \left(\frac{v_{\ell}}{r\cos{b}} \right)$',fontsize=14)
axs[1].set_ylabel(r'$\Delta\mu_b \,\,\, \left(\frac{v_{b}}{r\sin{b}}\right)$',fontsize=14)
axs[1].set_ylim(-150,150)
plt.show()

In [ ]:
cam2[0]